In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rarfile
import os

# Test de hipotesis para variables categoricas-categoricas
from scipy.stats import chi2_contingency

Sprint 1: Análisis de carácteristicas relevantes para predecir el target 

Cargamos el dataset y filtramos por tipos de datos

In [5]:
import zipfile
import os

ruta_archivo = "../../data/csv/df_balanced.zip"
directorio_extraccion = "../../data/csv/"

# Extraer el archivo ZIP
with zipfile.ZipFile(ruta_archivo, 'r') as archivo_zip:
    archivo_zip.extractall(directorio_extraccion)

# Obtener la ruta al archivo CSV extraído
archivo_csv_extraido = os.path.join(
    directorio_extraccion, 'df_balanced.csv')

In [8]:
df_train = pd.read_csv(archivo_csv_extraido)
df_train = df_train.drop(columns="Unnamed: 0")


df_train.head(5)

,auto_propio,casa_depto_propio,quien_acompañó,dia_inicio_proceso,telefono_trabajo,telefono_casa,telefono_casa2,reg_residencia_diferente,reg_trabajo_diferente,city_residencia_diferente,...,edad_cliente,n_familiares,obs_30_circulo_social,solicitudes_al_bureau,n_hijos,ingresos_totales,prestamo_anual,precio_bienes,anios_empleado,target
0,False,False,1,1,True,False,True,False,False,False,...,0.172307,2.0,1.0,0.0,0,0.426787,0.592702,1.600949,-0.477823,0
1,True,True,0,1,True,True,True,False,False,False,...,0.673812,1.0,0.0,0.0,0,-0.427191,-1.404683,-1.092160,-0.483056,0
2,False,True,0,3,True,False,False,False,False,False,...,0.673812,2.0,2.0,1.0,0,-0.142532,0.177887,-0.653464,-0.464739,0
3,False,True,0,4,True,False,False,False,False,False,...,0.924564,1.0,0.0,0.0,0,-0.199464,-0.361746,-0.068535,-0.464739,0
4,False,True,1,3,True,True,True,False,False,False,...,0.172307,2.0,0.0,1.0,0,-0.294350,0.028230,-0.226953,-0.475206,0


Filtrado de columnas categoricas para posterior aplicación de test de Chi-Cuadrado

In [4]:
total_cols = df_train.columns  # Total cols


cols_int = df_train.select_dtypes(include="int64")  # Cols integer

cols_bool = cols_int.columns[(

    df_train[cols_int.columns].isin([0, 1])).all()]  # Cols booleanas

cools_bool_df = df_train[cols_bool]  # DF Cols bool

cols_string = df_train.select_dtypes(include="object")  # Cols string


# Guardamos en listas ambas columnas
list_string = cols_string.columns.to_list()

list_boolean = cools_bool_df.columns.to_list()


# Columnas categoricas

cat_cols = list_string+list_boolean


print("Columnas totales:", len(total_cols))

print("")

print("Columnas booleanas:", len(cols_bool))

print("Columnas string:", cols_string.shape[1])

print("Columnas categoricas a evaluar:", len(cat_cols))


print("Columnas numericas :", len(total_cols)-(len(cat_cols)))


# Creamos un df de las categoricas solo para visualizar


df_cat = df_train[cat_cols]

Columnas totales: 122

Columnas booleanas: 33
Columnas string: 16
Columnas categoricas a evaluar: 49
Columnas numericas : 73


Test de Chi-Cuadrado para variables categoricas vs categoricas

In [5]:
resultados = []  # Creamos una lista para almacenar


for feature in cat_cols:
    # Primero calculamos la tabla de contigencia
    tabla_contingencia = pd.crosstab(df_cat[feature], df_train["TARGET"])
    # Realizamos el test
    chi2, p_valor, dof, expected = chi2_contingency(tabla_contingencia)

    # Guardamos los resultados
    resultados.append({
        "Variable": feature,
        "Chi-cuadrado": chi2,
        "P-value": p_valor,
        "Grados de libertad": dof,
        "Frecuencia esperadas": expected})

# Transformamos la lista de resultados en un dataframe
resultados_df = pd.DataFrame(resultados)

# Creamos una columna que nos facilite la interpretacion de P-value.
resultados_df["Significativa"] = np.where(
    resultados_df["P-value"] < 0.05, "Si", "No")

Hipotesis nula H0 : No existe relacion entre la variable y el target.
Hipotesis alternativa H1 : Existe relacion entre variables, no son independientes.

Si P-value > 0.05, se rechaza la hipotesis nula y se acepta relación entre variables.

Visualizamos los resultados por variables significativas y no significativas

In [6]:
non_sig = resultados_df.query("Significativa == 'No'")
non_sig = non_sig.reset_index(drop=True)

cat_significativas = resultados_df.query("Significativa=='Si'")
cat_significativas = cat_significativas.reset_index(drop=True)

Importante indicar que debido a la no explicación de las variables "FLAG_DOCUMENT", a pesar de tener significancia estadística, no serán consideradas para el analisis y la modelación ya que no es posible deducir a que documento corresponde y además, esto ayuda a reducir la dimensionalidad del modelo.

In [7]:
print(
    f"Luego de la evaluación de caracteristicas categoricas, pasaremos de considerar {df_cat.shape[1]} features, a {cat_significativas.shape[0]}")

Luego de la evaluación de caracteristicas categoricas, pasaremos de considerar 49 features, a 36


Ahora podemos extraer la lista con las columnas seleccionadas a partir del test

In [8]:
# Filtramos el df train para quedarnos solo con las categorias relevantes, la Primary Key y el target
cats = cat_significativas["Variable"].tolist()
cats.insert(0, "SK_ID_CURR")  # Insertamos el ID para el merge
df_cat_final = df_train[cats]


# Ordenamos para que el target siga segundo
target = df_cat_final["TARGET"]
df_cat_final = df_cat_final.drop(columns="TARGET")
df_cat_final.insert(1, "TARGET", target)

# Hacemos el drop de las columnas FLAGS
flags_to_drop = ["FLAG_DOCUMENT_2", "FLAG_DOCUMENT_3", "FLAG_DOCUMENT_6", "FLAG_DOCUMENT_8", "FLAG_DOCUMENT_9", "FLAG_DOCUMENT_11", "FLAG_DOCUMENT_13",
                 "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_18"]

df_cat_final = df_cat_final.drop(columns=flags_to_drop)
df_cat_final

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY
0,100002,1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,...,"Stone, brick",No,1,0,1,0,0,0,0,0
1,100003,0,Cash loans,F,N,N,Family,State servant,Higher education,Married,...,Block,No,1,0,1,0,0,0,0,0
2,100004,0,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,...,NaN,NaN,1,1,1,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,...,NaN,NaN,1,0,0,0,0,0,0,0
4,100007,0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,...,NaN,NaN,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,...,"Stone, brick",No,1,0,0,0,0,0,0,0
307507,456252,0,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,...,"Stone, brick",No,0,0,1,0,0,0,0,0
307508,456253,0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,...,Panel,No,1,0,0,0,0,0,1,1
307509,456254,1,Cash loans,F,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,...,"Stone, brick",No,1,0,0,0,0,1,1,0


In [9]:
# Calculamos un dummies tentativo, para ver como aumenta la dimesionalidad
first_shape_cat = df_cat_final.shape[1]

first_dummies = pd.get_dummies(df_cat_final)
first_shape = first_dummies.shape[1]

Renombramos el dataset categórico final

In [11]:

old_names = df_cat_final.columns  # Los nombres del df sin tranformar

# Diccionario para renombrar las columnas categoricas

dict_cat = {'SK_ID_CURR': "sk_id_curr",
            'TARGET': "target",
            'NAME_CONTRACT_TYPE': "tipo_contrato",
            'CODE_GENDER': "genero",
            'FLAG_OWN_CAR': "auto_propio",
            'FLAG_OWN_REALTY': "casa_depto_propio",
            'NAME_TYPE_SUITE': "quien_acompañó",  # Al momento de solicitar el prestamo
            'NAME_INCOME_TYPE': "estatus_laboral",
            'NAME_EDUCATION_TYPE': "nivel_educacion",
            'NAME_FAMILY_STATUS': "estado_civil",
            'NAME_HOUSING_TYPE': "forma_habitar",
            'OCCUPATION_TYPE': "ocupacion",
            'WEEKDAY_APPR_PROCESS_START': "dia_inicio_proceso",
            'ORGANIZATION_TYPE': "tipo_organizacion_trabajo",
            'FONDKAPREMONT_MODE': "tipo_cuenta_bancaria",
            # Normalizada, tipo de vivienda donde el cliente vive, es un promedio
            'HOUSETYPE_MODE': "tipo_vivienda",
            'WALLSMATERIAL_MODE': "material_estructural",
            'EMERGENCYSTATE_MODE': "estado_emergencia",
            'FLAG_EMP_PHONE': "telefono_trabajo",  # Si lo entregó al momento de solicitar
            'FLAG_WORK_PHONE': "telefono_casa",
            'FLAG_PHONE': "telefono_casa2",  # Este podria ser lo mismo que emp phone
            # Si la direccion de residencia no coincide con la direccion de contacto
            'REG_REGION_NOT_LIVE_REGION': "reg_residencia_diferente",
            # Si la direccion de residencia no coincide con la direccion de trabajo
            'REG_REGION_NOT_WORK_REGION': "reg_trabajo_diferente",
            # Si la direccion de residencia no coincide con la direccion de contacto
            'REG_CITY_NOT_LIVE_CITY': "city_residencia_diferente",
            # Si la direccion de residencia no coincide con la direccion de trabajo
            'REG_CITY_NOT_WORK_CITY': "city_trabajo_diferente",
            # Si la direccion de residencia no coincide con la direccion de trabajo
            'LIVE_CITY_NOT_WORK_CITY': "live_trabajo_diferente",
            }

df_cat_final = df_cat_final.rename(columns=dict_cat)

Sprint 2. Preprocesamiento dataset A

Revision y preprocesamiento de variables categoricas

In [13]:
def data_quality(df, df_name):
    # Dimension
    print("======= Revision de dimension ==========")
    print("")
    print(f"El shape de {df_name} es {df.shape}")
    print("")
    # Revision de valores nulls
    print("======= Revision de nulos ==========")
    print("")
    print(f"Cantidad de nulos por columna en el dataset {df_name}:")
    print("")
    for colname in df.columns:
        col_nulls = df[colname].isna().sum()
        print(f"{colname}: {col_nulls}")
    print("")

    # Revision dtypes
    print("")
    print("======= Revision dtypes ==========")
    print("")
    print(df.dtypes)

    # Revision de duplicados
    print("")
    print("======= Revision duplicados ==========")
    print("")
    print(f"Cantidad de duplicados en {df_name} : {df.duplicated().sum()}")

    # Revision de valores unicos, con esto podemos ver si las variables categoricas presentan valores erroneos, ya que los outliers solo aplican para numericas.
    print("")
    print("======= Revision valores unicos ==========")
    print("")
    for colname in df.columns:
        col_nunique = df[colname].nunique()
        print(f"{colname}: {col_nunique}")
    print("")

    # Valores unicos variables categoricas
    print("")
    print("======= Revision valores unicos categoricas ==========")
    print("")
    for colname in df.columns:
        if df[colname].dtypes not in ["int", "float"]:
            col_unique = df[colname].unique()
            print(f"Columna {colname}:")
            print("")
            print(col_unique)
            print("")

Revision calidad de datos

In [14]:
data_quality(df_cat_final, "Dataframe Variables categoricas")

======= Revision de dimension ==========

El shape de Dataframe Variables categoricas es (307511, 26)

======= Revision de nulos ==========

Cantidad de nulos por columna en el dataset Dataframe Variables categoricas:

sk_id_curr: 0
target: 0
tipo_contrato: 0
genero: 0
auto_propio: 0
casa_depto_propio: 0
quien_acompañó: 1292
estatus_laboral: 0
nivel_educacion: 0
estado_civil: 0
forma_habitar: 0
ocupacion: 96391
dia_inicio_proceso: 0
tipo_organizacion_trabajo: 0
tipo_cuenta_bancaria: 210295
tipo_vivienda: 154297
material_estructural: 156341
estado_emergencia: 145755
telefono_trabajo: 0
telefono_casa: 0
telefono_casa2: 0
reg_residencia_diferente: 0
reg_trabajo_diferente: 0
city_residencia_diferente: 0
city_trabajo_diferente: 0
live_trabajo_diferente: 0


======= Revision dtypes ==========

sk_id_curr                    int64
target                        int64
tipo_contrato                object
genero                       object
auto_propio                  object
casa_depto_propio    

Luego de la revisión de calidad de datos, tomaremos las siguientes decisiones con respecto a las columnas con muchos nulos

-quien_acompañó: 1292 , Imputaremos esta columna con "Unaccompanied" 
-ocupacion: 96391 , Imputaremos con "Otros"
-tipo_cuenta_bancaria: 210295 Imputaremos con "not_specified"
-tipo_vivienda: 154297 , con más del 50% de nulos y unas caracteristicas que podrían generar sesgo, se eliminará la caracteristica
-material_estructural: 156341 , con más del 50% de nulos y unas caracteristicas que podrían generar sesgo, se eliminará la caracteristica
-estado_emergencia: 145755 , con más del 50% de nulos y unas caracteristicas que podrían generar sesgo, se eliminará la caracteristica
-A pesar de que Genero es una variable relevante para el target, no la consideraremos con el objetivo de disminuir el sesgo del modelo frente al Género, para no caer en discriminaciones arbitrarias.

Transformaciones para reducir dimensionalidad

In [15]:
# Rellenamos los nulos y dropeamos las columnas anteriormente comentadas
df_cat_final["quien_acompañó"] = df_cat_final["quien_acompañó"].fillna(
    "Unaccompanied")

# Hacemos booleana la variables de si fue acompañado o no, debido a que existen muchas sub categorias con poca representación.
dict_accom = {"Other_A": 1,
              "Other_B": 1,
              'Spouse, partner': 1,
              'Children': 1,
              'Family': 1,
              'Group of people': 1,
              'Unaccompanied': 0
              }

df_cat_final["quien_acompañó"] = df_cat_final["quien_acompañó"].replace(
    dict_accom).astype(int)


# Rellenamos algunos nulos, imputandolos con otros valores
df_cat_final["ocupacion"] = df_cat_final["ocupacion"].fillna("Other")
df_cat_final["tipo_cuenta_bancaria"] = df_cat_final["tipo_cuenta_bancaria"].fillna(
    "not specified")

# Dropeamos las columnas que no utilizaremos por las razones comentadas más arriba.
df_cat_final = df_cat_final.drop(
    columns=["tipo_vivienda", "material_estructural", "estado_emergencia", "genero"])


# Transformamos las variables a boolean
df_cat_final["telefono_trabajo"] = df_cat_final["telefono_trabajo"].astype(
    bool)
df_cat_final["telefono_casa"] = df_cat_final["telefono_casa"].astype(bool)
df_cat_final["telefono_casa2"] = df_cat_final["telefono_casa2"].astype(bool)
df_cat_final["reg_residencia_diferente"] = df_cat_final["reg_residencia_diferente"].astype(
    bool)
df_cat_final["reg_trabajo_diferente"] = df_cat_final["reg_trabajo_diferente"].astype(
    bool)
df_cat_final["city_residencia_diferente"] = df_cat_final["city_residencia_diferente"].astype(
    bool)
df_cat_final["city_trabajo_diferente"] = df_cat_final["city_trabajo_diferente"].astype(
    bool)
df_cat_final["live_trabajo_diferente"] = df_cat_final["live_trabajo_diferente"].astype(
    bool)

# Columna auto propio
dic_boolean_1 = {"Y": True, "N": False}
df_cat_final["auto_propio"] = df_cat_final["auto_propio"].replace(
    dic_boolean_1).astype(bool)
# Columna depto propio
df_cat_final["casa_depto_propio"] = df_cat_final["casa_depto_propio"].replace(
    dic_boolean_1).astype(bool)


# Transformamos los dias en numeros, para disminuir dimension de dias
dict_days = {"MONDAY": 1, "TUESDAY": 2, "WEDNESDAY": 3,
             "THURSDAY": 4, "FRIDAY": 5, "SATURDAY": 6, "SUNDAY": 7}
df_cat_final["dia_inicio_proceso"] = df_cat_final["dia_inicio_proceso"].replace(
    dict_days).astype(int)


# Transformamos los 2 registros desconocidos a la categoria más generica en estado civil
dict_civil = {"Unknown": "Single / not married"}
df_cat_final["estado_civil"] = df_cat_final["estado_civil"].replace(dict_civil)


# Unificamos algunas subcategorias en estatus laboral
dict_laboral = {'State servant': 'Working',
                'Commercial associate': 'Working',
                'Businessman': 'Working',
                'Maternity leave': 'Working',
                'Student': 'Unemployed'}  # Si no genera ingresos, se cataloga como desempleado, a pesar de que sea estudiante.

df_cat_final["estatus_laboral"] = df_cat_final["estatus_laboral"].replace(
    dict_laboral)

# Reducciones en columna ocupacion
dict_ocupation = {
    "IT staff": "Other",
    "HR staff": "Other",
    "Private service staff": "Other",
    "Low-skill Laborers": "Other",
    "Waiters/barmen staff": "Other",
    "Secretaries": "Other",
    "Realty agents": "Other",
    "Cooking staff": "Other",
    "Cleaning staff": "Other",
    "High skill tech staff": "Other",
    "Accountants": "Other",
    "Medicine staff": "Other",
    "Security staff": "Other"
}

df_cat_final["ocupacion"] = df_cat_final["ocupacion"].replace(dict_ocupation)


# Reducciones en columna tipo de organizacion
dict_orgs = {
    # Real state
    "Housing": 'Real Estate',
    "Realtor": "Real Estate",

    # Construction
    'Electricity': "Construction",
    'Telecom': "Construction",

    # Education
    'Kindergarten': "Education",
    'University': "Education",
    'School': "Education",

    # Armed forces
    'Military': "Armed_forces",
    'Police': "Armed_forces",

    # Government
    'Security Ministries': 'Government',

    # Business
    'Business Entity Type 1': "Business",
    'Business Entity Type 2': "Business",
    'Business Entity Type 3': "Business",

    # Industry
    'Industry: type 1': "Industry",
    'Industry: type 10': "Industry",
    'Industry: type 11': "Industry",
    'Industry: type 12': "Industry",
    'Industry: type 13': "Industry",
    'Industry: type 2': "Industry",
    'Industry: type 3': "Industry",
    'Industry: type 4': "Industry",
    'Industry: type 5': "Industry",
    'Industry: type 6': "Industry",
    'Industry: type 7': "Industry",
    'Industry: type 8': "Industry",
    'Industry: type 9': "Industry",

    # Trade
    'Trade: type 1': "Trade",
    'Trade: type 2': "Trade",
    'Trade: type 3': "Trade",
    'Trade: type 4': "Trade",
    'Trade: type 5': "Trade",
    'Trade: type 6': "Trade",
    'Trade: type 7': "Trade",

    # Transport
    'Transport: type 1': "Transport",
    'Transport: type 2': "Transport",
    'Transport: type 3': "Transport",
    'Transport: type 4': "Transport",

    # Finance
    'Business': "Finance/Business",
    'Bank': "Finance/Business",
    "Insurance": "Finance/Business",

    # Otros, debido a que representan menos de 1% de los datos, no se considerarán relevantes y se agruparán.
    'XNA': "Other",
    'Advertising': "Other",
    'Agriculture': "Other",
    "Religion": "Other",
    "Cleaning": "Other",
    'Culture': "Other",
    "Mobile": "Other",
    'Emergency': "Other",
    'Hotel': "Other",
    'Legal Services': "Other",
    'Postal': "Other",
    'Restaurant': "Other",
    'Security': "Other",
    "Services": "Other",
}

# La aplicacion de esta transformacion en la columna, redujo de 59 a 13 valores unicos, lo que ayudará a controlar la dimensionalidad.
df_cat_final["tipo_organizacion_trabajo"] = df_cat_final["tipo_organizacion_trabajo"].replace(
    dict_orgs)


# Columna estado_civil
dict_civil = {'Civil marriage': 'Married'}

df_cat_final["estado_civil"] = df_cat_final["estado_civil"].replace(dict_civil)


# Columna tipo de cuenta bancaria, se unificarán por cuentas personales y empresa, a pesar de que el 70% de la columna no especifica el tipo
dict_cuenta = {
    "reg oper account": "personal_account",
    "reg oper spec account": "personal_account",
    "org spec account": "business_account"}

df_cat_final["tipo_cuenta_bancaria"] = df_cat_final["tipo_cuenta_bancaria"].replace(
    dict_cuenta)

# Columna forma de habitar
dict_hab = {'Municipal apartment': 'Rented apartment',
            'Office apartment': "Office/Co-op apartment",
            'Co-op apartment': "Office/Co-op apartment"}

df_cat_final["forma_habitar"] = df_cat_final["forma_habitar"].replace(dict_hab)

In [ ]:
    # Valores unicos variables categoricas
    for colname in df_cat_final.columns:
            col_unique = df_cat_final[colname].unique()
            print(f"Columna {colname}:")
            print("")
            print(col_unique)
            print("")

Revision final dimensionalidad antes del modelamiento

In [17]:
# Shape del dataset sin preprocesar, solo significativas
old = df_cat_final.shape[1]
test_dummies = pd.get_dummies(df_cat_final)  # Obtenemos los dummies


new = test_dummies.shape[1]  # df con dummies aplicados


diferencia = first_shape-new
percent = (diferencia/first_shape) * 100


print(
    f"Columnas Dataset categorico, solo significativas sin preprocesar {first_shape_cat}")


print(
    f"Columnas Dataset categorico, solo significativas sin preprocesar aplicando dummies {first_shape}")


print(f"Columnas Dataset categorico preprocesado antes de dummies {old}")


print(f"Columnas Dataset categorico preprocesado con dummies {new}")

print(
    f"Luego del preprocesamiento de datos se logró disminuir la dimensionalidad final del dataset categorico en % {int(percent)}")

Columnas Dataset categorico, solo significativas sin preprocesar 26
Columnas Dataset categorico, solo significativas sin preprocesar aplicando dummies 150
Columnas Dataset categorico preprocesado antes de dummies 22
Columnas Dataset categorico preprocesado con dummies 54
Luego del preprocesamiento de datos se logró disminuir la dimensionalidad final del dataset categorico en % 64


Unificamos las variables numericas y categoricas en un solo dataset.

In [18]:
df_num = pd.read_csv("data_num.csv")
df_num = df_num.drop(columns="Unnamed: 0")

In [19]:
new_names = {"MONTO_CREDITO": "monto_credito",
             "ANIOS_EDAD": "edad_cliente",
             "CANT_MIEMBROS_FAM": "n_familiares",
             "OBS_30_CNT_CIRCULO_SOCIAL": "obs_30_circulo_social",
             "MONTO_REQ_CREDITO_ANUAL": "solicitudes_al_bureau",
             "ID_CLIENTE": "sk_id_curr",
             "CANT_HIJOS": "n_hijos",
             "MONTO_INGRESO_TOTAL": "ingresos_totales",
             "ANUALIDAD_MENSUAL": "prestamo_anual",
             "PRECIO_BIENES": "precio_bienes",
             "ANIOS_EMPLEADO": "anios_empleado"}


df_num = df_num.rename(columns=new_names)
df_concat = test_dummies.merge(df_num, how="inner", on="sk_id_curr")

# Exportamos el df concatenado y limpiado para la modelación
df_concat.to_csv("df_concat_a.csv")